# AgentsVille Trip Planner - Project Assignment

In this project, you'll implement an AI system to help you plan a trip--to the wonderful city of AgentsVille!

Your AI system will demonstrate advanced LLM reasoning techniques including:

1. **Role-Based Prompting** - Your agent will act as a specialized travel planner
2. **Chain-of-Thought Reasoning** - Step-by-step planning of itineraries
3. **ReAct Prompting** - Thought → Action → Observation cycles
4. **Feedback Loops** - Self-evaluation using tools in the ReAct loop to find mistakes and improve plans

You'll simulate external API calls to gather weather data and activities. Then, process this information to create personalized travel itineraries based on interests and other constraints. Last, you'll implement a feedback loop to refine the plan.

Your task is to build a travel agent that can plan the perfect AgentsVille vacation!

## Project Instructions

Here are the steps you'll follow:

1. **Define Vacation Details**
    - Specify the trip duration, interests, and constraints.
    - Use Pydantic to structure and verify this information in a class called `VacationInfo`.
2. **Review Weather and Activity schedules**
    - Simulate API calls to gather weather data and available activities in bulk
    - Review the data manually to understand the available options
3. **The ItineraryAgent**
    - Implement an agent that generates a day-by-day itinerary based on the vacation details
    - The system prompt will guide the agent's reasoning through a step-by-step planning process to take travel preferences (e.g. destination, dates, interests) and generate a detailed day-by-day itinerary
    - Craft the components of the prompt (including the role, task/instructions, output format, examples, and context) to elicit the best possible itinerary in one LLM call
4. **Evaluating the Itinerary**
    - Evaluate the itinerary using a set of criteria to ensure a high-quality travel plan
        - For instance, does the itinerary match the city and the dates requested?
        - Or, is the total cost calulation accurate and is it within budget?
        - Or, does the agent hallucinate any activities that are not available?
        - Or, does the agent suggest activities that are not suitable for the weather? This specific evaluation function will require the use of an LLM to compare the event description against the weather data.
5. **Defining the Tools**
    - We will define four tools to assist the agent
        - **calculator_tool**: to accurately calculate costs
        - **get_activities_by_date_tool**: to retrieve activities for a specific date
        - **run_evals_tool**: to evaluate the itinerary against the criteria
        - **final_answer_tool**: to provide the final answer in a structured format
6. **The ItineraryRevisionAgent**
    - We will implement a second agent that revises the itinerary based on feedback using the ReAct THOUGHT → ACTION → OBSERVATION cycle.
        - The LLM will first generated a THOUGHT / ACTION message, which contains reasoning steps and a tool call invocation.
        - The Python code will parse the tool call and execute it, returning the result as a string to the LLM in an OBSERVATION message.
        - After this cycle repeats n number of times, the LLM will invoke the final_answer_tool to signal to the Python code to end the loop and return the final answer.
    - This agent will also **incorporate feedback on the initial itinerary** from the travelers to ensure the final plan has **at least 2 activities per day**. A new evaluation function using a powerful LLM will be created to check this user feedback.
    - The agent will use the tools above to refine the plan iteratively, checking the weather and available activities, and ensuring the itinerary meets all constraints.
7. **Something just for fun!**
    - To wrap things up we'll create a fun, narrative summary of the trip, highlighting the best activities and experiences!

## Initial Setup

Let's start with settin up our environment and defining the vacation details.

In [16]:
# When using VSCode in the Udacity workspace, add /workspace to the PYTHON_PATH
import os
import sys

WORKSPACE_DIRECTORY = "/workspace"
if os.path.exists(WORKSPACE_DIRECTORY) and WORKSPACE_DIRECTORY not in sys.path:
    sys.path.append(WORKSPACE_DIRECTORY)
    print(f"Added {WORKSPACE_DIRECTORY} to the Python path")

In [17]:
# Install required packages if not already installed
# No changes needed here.
%pip install -q json-repair==0.47.1 numexpr==2.11.0 openai==1.74.0 pandas==2.3.0 pydantic==2.11.7 python-dotenv==1.1.0


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
# If using the Vocareum API endpoint
# TODO: Fill in the missing parts marked with **********
from openai import OpenAI

# client = OpenAI(
#     # Change the base_url when using the Vocareum API endpoint
#     # If using the OpenAI API endpoint, you can comment out the base_url line
#     base_url="https://openai.vocareum.com/v1",
#     # Uncomment one of the following
#     api_key="**********",  # <--- TODO: Fill in your Vocareum API key here
#     # api_key=os.getenv(
#     #     "OPENAI_API_KEY"
#     # ),  # <-- Alternately, set as an environment variable
# )
from dotenv import load_dotenv
load_dotenv()  
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [19]:
# Throughout this project you can experiment with different OpenAI models.
# By default we will use GPT-4.1-mini, which is a good balance of speed and cost.
from enum import Enum

class OpenAIModel(str, Enum):
    GPT_41 = "gpt-4.1"  # Strong default choice for development tasks, particularly those requiring speed, responsiveness, and general-purpose reasoning. 
    GPT_41_MINI = "gpt-4.1-mini"  # Fast and affordable, good for brainstorming, drafting, and tasks that don't require the full power of GPT-4.1.
    GPT_41_NANO = "gpt-4.1-nano"  # The fastest and cheapest model, suitable for lightweight tasks, high-frequency usage, and edge computing.

MODEL = OpenAIModel.GPT_41_MINI  # Default model for this project


## Define Vacation Details

Let's encode the details of our vacation in JSON format and verify it using Pydantic.

In practice, a chatbot agent could gather the information of a user. After it has gathered all the information it needs, it could generate this JSON object from the chat transcript. We will skip that step to focus on the itinerary generation itself.

In [20]:
# The Vacation Info Data Structure
# No changes needed here, but you may choose to personalize the data.

VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Yuri",
            "age": 30,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["tennis", "cooking", "comedy", "technology"],
        },
        {
            "name": "Hiro",
            "age": 25,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["reading", "music", "theatre", "art"],
        },
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",  # Mock data exists for 2025-06-10
    "date_of_departure": "2025-06-12",  # ...until 2025-06-15.
    "budget": 130,  # Budget is in fictional currency units.
}

In [21]:
# Validate the data structure using Pydantic
# COMPLETED: VacationInfo class definition with Pydantic

# Pydantic allows us to automatically validate data and ensure correct structure
# It acts as a "contract" specifying which fields an object must have and their types

from project_lib import Interest
from typing import List
from pydantic import BaseModel
import datetime
from pprint import pprint

class Traveler(BaseModel):
    """A traveler with name, age, and list of interests.

    Attributes:
        name (str): Traveler name.
        age (int): Traveler age.
        interests (List[Interest]): List of traveler interests.
    """
    name: str
    age: int
    interests: List[Interest]

class VacationInfo(BaseModel):
    """Vacation information including travelers, destination, dates, and budget.

    This class automatically validates that all fields have the correct type.
    If you try to create a VacationInfo with invalid data, Pydantic will raise an error.

    Attributes:
        travelers (List[Traveler]): List of travelers.
        destination (str): Vacation destination.
        date_of_arrival (datetime.date): Arrival date.
        date_of_departure (datetime.date): Departure date.
        budget (int): Budget for vacation in fictional currency units.
    """
    # COMPLETED: The 5 required fields for VacationInfo
    travelers: List[Traveler]  # List of travelers (each is a Traveler object)
    destination: str  # Destination name (string)
    date_of_arrival: datetime.date  # Arrival date (Python date object)
    date_of_departure: datetime.date  # Departure date (Python date object)
    budget: int  # Total budget (integer)


# Validate the VacationInfo data structure
vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)

# Display the vacation info as a dictionary
pprint(vacation_info.model_dump())

# Check that VacationInfo contains the expected data
assert "travelers" in vacation_info.model_dump().keys()
assert "destination" in vacation_info.model_dump().keys()
assert "date_of_arrival" in vacation_info.model_dump().keys()
assert "date_of_departure" in vacation_info.model_dump().keys()
assert "budget" in vacation_info.model_dump().keys()
assert isinstance(vacation_info.travelers, list)
assert all(isinstance(traveler, Traveler) for traveler in vacation_info.travelers)
assert isinstance(vacation_info.date_of_arrival, datetime.date)
assert isinstance(vacation_info.date_of_departure, datetime.date)
assert isinstance(vacation_info.budget, int)

print("[OK] VacationInfo data structure is valid!")

{'budget': 130,
 'date_of_arrival': datetime.date(2025, 6, 10),
 'date_of_departure': datetime.date(2025, 6, 12),
 'destination': 'AgentsVille',
 'travelers': [{'age': 30,
                'interests': [tennis, cooking, comedy, technology],
                'name': 'Yuri'},
               {'age': 25,
                'interests': [reading, music, theatre, art],
                'name': 'Hiro'}]}
[OK] VacationInfo data structure is valid!


## Review Weather and Activity Schedules

Now that we have the trip details, we can retrieve the weather and activity schedules for the dates of the trip.

We will call an API to get all the data at once, in order to be able to include it in the context for our itinerary planning agent.

Also, we will format the data as Pandas DataFrames for easier viewing. Take the time to read and understand the data to see if the agent notices the same things you do. For instance:
- What does the weather look like for the trip? On what days it is sunny, rainy, or cloudy?
- What activities are available on each day? Are there any special events or festivals related to the user's interests?

In [22]:
# call_weather_api_mocked simulates calling a weather API to get weather data
# COMPLETED: Get weather data for all trip dates

from project_lib import call_weather_api_mocked
import pandas as pd

pd.set_option("display.max_colwidth", None)

# EXPLANATION: We need to get weather for each day of the trip
# pd.date_range creates a sequence of dates from arrival to departure
# For each date, we call the mock weather API

weather_for_dates = [
    call_weather_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
    for ts in pd.date_range(
        # COMPLETED: Use arrival and departure dates from vacation_info
        start=vacation_info.date_of_arrival,  # Trip start date
        end=vacation_info.date_of_departure,  # Trip end date
        freq="D",  # "D" means "daily" - one date per day
    )
]

# Convert to DataFrame for clearer visualization
weather_for_dates_df = pd.DataFrame(weather_for_dates)

weather_for_dates_df

,date,city,temperature,temperature_unit,condition,description
0,2025-06-10,AgentsVille,31,celsius,clear,A bright and sunny day in AgentsVille with clear skies and warm temperatures. Perfect weather for outdoor activities!
1,2025-06-11,AgentsVille,34,celsius,partly cloudy,"A warm day with periods of sunshine and mixed clouds, making it a perfect opportunity for outdoor activities."
2,2025-06-12,AgentsVille,28,celsius,thunderstorm,"A thunderstorm is expected to roll in during the afternoon, bringing heavy rain and gusty winds. The atmosphere will feel charged with humidity, creating a sultry and dramatic setting as clouds build in the sky."


In [23]:
# call_activities_api_mocked returns activities for a given date and city
# COMPLETED: Get available activities for all trip dates

from project_lib import call_activities_api_mocked

# EXPLANATION: Similar to weather, we need to get all available activities
# during our trip. This information is crucial for the agent to plan
# the itinerary considering which activities are actually available each day.

activities_for_dates = [
    activity
    for ts in pd.date_range(
        # COMPLETED: Same dates as for weather
        start=vacation_info.date_of_arrival,  # First trip date
        end=vacation_info.date_of_departure,  # Last trip date
        freq="D",  # Daily frequency
    )
    # For each date, get ALL available activities that day
    # and "flatten" them into a single list
    for activity in call_activities_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
]

# Convert to DataFrame to see all activities in organized format
activities_for_dates_df = pd.DataFrame(activities_for_dates)

activities_for_dates_df

,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00,2025-06-10 11:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[technology]
1,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00,2025-06-10 13:30,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[cooking, tennis]"
2,event-2025-06-10-2,Artful Athletics: Paint & Play Extravaganza,2025-06-10 15:00,2025-06-10 17:00,"Creative Courts Park, AgentsVille","Join us for an exciting afternoon at Creative Courts Park, where the worlds of art and sports collide! At 'Artful Athletics: Paint & Play Extravaganza', you'll participate in collaborative outdoor murals inspired by your favorite sports, and then get moving with fun, friendly sports mini-games. Whether you love painting or playing, this event celebrates creativity, teamwork, and the joy of movement under the open sky. Perfect for art lovers and sports enthusiasts alike—come ready to express yourself and get active! (Event is held outdoors; in case of rain, we move indoors to the Community Gym nearby.)",15,"[art, sports]"
3,event-2025-06-10-3,AgentsVille Twilight Writing Escape,2025-06-10 19:00,2025-06-10 21:00,"The Ink Loft, 12 Quill Lane, AgentsVille","Join fellow writers for an inspiring evening at The Ink Loft, where words flow as freely as the coffee! This writing-themed event welcomes all—novelists, poets, bloggers, or anyone with a passion for storytelling. Set indoors in AgentsVille's coziest lounge, enjoy writing games, group prompts, and opportunities to read your work aloud. Connect, create, and celebrate the art of writing in this creative indoor haven.",15,"[writing, reading, art]"
4,event-2025-06-11-0,Morning Groove Dance Party,2025-06-11 09:00,2025-06-11 10:30,"Rhythm Hall, Center Plaza, AgentsVille","Start your day with energy and joy at the Morning Groove Dance Party! This lively event welcomes dancers of all levels to join a vibrant indoor session filled with upbeat music and fun routines. Whether you love modern pop, Latin beats, or classic disco, our dance instructors will guide you to move and groove. Connect with fellow dance lovers in the colorful atmosphere of Rhythm Hall. Perfect for fans of dancing, music, and fitness. Let the rhythm move you! (Indoor event.)",15,"[dancing, music, fitness]"
5,event-2025-06-11-1,Tech Lunch & Learn: AI Frontiers,2025-06-11 12:00,2025-06-11 13:30,"The Digital Atrium, AgentsVille","Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect with others passionate about technology, AI, and the digital world. Whether you're a seasoned developer or just curious about tech, this event is for you! Related interests: technology, music (sound tech demos), photography (AI imaging), writing (AI cr

## The ItineraryAgent

First we will review the Pydantic objects used for defining the output of our agent, the TravelPlan, ItineraryDay, Activity, and Weather classes.

Second, we will create a Chain-of-Thought prompt to guide the agent in planning the trip. This prompt will instruct the agent to consider the weather, activities, and user preferences when creating the itinerary.

Third, we will run the agent to produce the TravelPlan object, which will will refine in the following steps.

In [24]:
# Review the data structure we will use for representing a TravelPlan, which includes
# weather, activity_recommendations, and itinerary for each day of the trip.
# Our goal is to take a VacationInfo object and return a TravelPlan object.
# No changes are needed here.

class Weather(BaseModel):
    temperature: float
    temperature_unit: str
    condition: str


class Activity(BaseModel):
    activity_id: str
    name: str
    start_time: datetime.datetime
    end_time: datetime.datetime
    location: str
    description: str
    price: int
    related_interests: List[Interest]


class ActivityRecommendation(BaseModel):
    activity: Activity
    reasons_for_recommendation: List[str]


class ItineraryDay(BaseModel):
    date: datetime.date
    weather: Weather
    activity_recommendations: List[ActivityRecommendation]


class TravelPlan(BaseModel):
    city: str
    start_date: datetime.date
    end_date: datetime.date
    total_cost: int
    itinerary_days: List[ItineraryDay]

In [25]:
# ENHANCED: Production-level prompt with advanced MLOps/AI Engineering techniques
# Original prompt critique:
# - Lacked specificity in success criteria
# - Did not handle edge cases explicitly
# - Could be more deterministic
# - Missing guidance on prioritization during conflicts
# IMPROVEMENTS APPLIED: SMART criteria, edge case handling, explicit prioritization

import json
from project_lib import ChatAgent
from typing import Optional

# MLOps/AI ENGINEER IMPROVEMENTS:
# 1. SMART criteria (Specific, Measurable, Achievable, Relevant, Time-bound)
# 2. Explicit edge case and conflict handling
# 3. Low temperature (0.2) for greater determinism
# 4. Clear output schema validation
# 5. Explicit constraint prioritization
# 6. Clear fallback strategies

ITINERARY_AGENT_SYSTEM_PROMPT = f"""You are an expert Itinerary Planning Agent with 10+ years of experience in travel planning and optimization.

## Task Objective

Generate a COMPLETE, VALID day-by-day travel itinerary that STRICTLY adheres to ALL constraints below.

## Mandatory Constraints (Must ALL be satisfied - ZERO tolerance for violations)

1. **Date Range**: Itinerary MUST span EXACTLY from arrival to departure dates (inclusive)
2. **Budget**: Total cost MUST be ≤ budget (NOT exceed even by 1 unit)
3. **Activity Existence**: ONLY use activities from "Available Activities" list (NO hallucinations)
4. **Activity Dates**: ONLY use activities available on the EXACT date being planned
5. **Daily Minimum**: AT LEAST 1 activity per day (preferably 1-3 activities)
6. **Weather Compatibility**:
   - AVOID outdoor-only activities on rainy days
   - EXCEPTION: If activity description explicitly mentions "indoor backup" or "covered area"
7. **Interest Matching**: AT LEAST 1 activity per traveler matching their interests across the entire trip

## Planning Process (Chain-of-Thought - Execute in THIS order)

### Phase 1: Data Analysis
```
STEP 1.1 - Parse Traveler Interests:
- List each traveler and their interests
- Identify overlapping interests (activities both enjoy)
- Note unique interests (need separate activities)

STEP 1.2 - Weather Analysis:
- Date | Condition | Suitable for outdoor?
- Identify "risky" days (rain/storms)
- Plan indoor-heavy itinerary for those days

STEP 1.3 - Activity Inventory:
- Total activities available: X
- Activities by interest category: [breakdown]
- Activities by weather suitability: [indoor vs outdoor]
- Price range: min=X, max=Y, avg=Z
```

### Phase 2: Budget Allocation Strategy
```
STEP 2.1 - Calculate Constraints:
- Total budget: B
- Number of days: N
- Recommended budget per day: B/N
- Must-have activities (high interest match): [list with costs]
- Remaining budget after must-haves: X

STEP 2.2 - Prioritization Rules:
Priority 1 (P1): Activities matching BOTH travelers' interests
Priority 2 (P2): Activities matching one traveler's interests + weather-appropriate
Priority 3 (P3): Unique experiences (festivals, special events)
Priority 4 (P4): Filler activities to meet minimum daily requirement
```

### Phase 3: Day-by-Day Planning
```
FOR EACH DAY:

STEP 3.1 - Filter Available Activities:
- Get activities available on THIS date
- Remove if: outdoor-only AND weather is rainy
- Remove if: cost would exceed remaining budget

STEP 3.2 - Select Activities:
- Choose 1-3 activities using prioritization (P1 > P2 > P3 > P4)
- Ensure time slots don't overlap
- Track running cost and interests covered
```

### Phase 4: Validation
```
Pre-submission Checks:
[ ] Total days match date range?
[ ] Every day has ≥1 activity?
[ ] Total cost ≤ budget?
[ ] Each traveler has ≥1 interest-matching activity?
[ ] No outdoor activities on rain days?
[ ] All activities exist in available list?
```

## Output Format

Respond with TWO sections - ANALYSIS then FINAL OUTPUT:

```
ANALYSIS:
[Your reasoning here]

FINAL OUTPUT:
```json
{json.dumps(TravelPlan.model_json_schema(), indent=2)}
```

## Context Data

### Weather Data:
{weather_for_dates_df.to_json(orient='records', indent=2)}

### Available Activities:
{activities_for_dates_df.to_json(orient='records', indent=2)}

## Temperature Settings
- temperature=0.2 for determinism
"""

assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper()
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper()

class ItineraryAgent(ChatAgent):
    """Production-grade agent with enhanced error handling."""
    system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT

    def get_itinerary(self, vacation_info: VacationInfo, model: Optional[OpenAIModel] = None) -> TravelPlan:
        """Generates travel itinerary with retry logic and JSON extraction fix."""
        from project_lib import print_in_box

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = (self.chat(
                    user_message=vacation_info.model_dump_json(indent=2),
                    add_to_messages=False,
                    model=model or self.model,
                    temperature=0.2,
                ) or "").strip()

                print_in_box(response, f"Raw Response (Attempt {attempt + 1})")

                # CRITICAL FIX: Enhanced JSON extraction
                json_text = response.strip()

                # Extract from FINAL OUTPUT section if present
                if 'FINAL OUTPUT:' in json_text:
                    json_text = json_text.split('FINAL OUTPUT:')[-1].strip()

                # Extract from code block
                if '```json' in json_text:
                    json_text = json_text.split('```json')[1].split('```')[0].strip()
                elif '```' in json_text:
                    parts = json_text.split('```')
                    if len(parts) >= 3:
                        json_text = parts[1].strip()

                # CRITICAL: Handle wrapped JSON (ANALYSIS key or FINAL OUTPUT key)
                import json as json_module
                try:
                    parsed = json_module.loads(json_text)
                    if isinstance(parsed, dict):
                        # Remove wrapper keys if present
                        if 'FINAL OUTPUT' in parsed:
                            parsed = parsed['FINAL OUTPUT']
                            json_text = json_module.dumps(parsed)
                        elif 'ANALYSIS' in parsed:
                            parsed.pop('ANALYSIS', None)
                            json_text = json_module.dumps(parsed)
                except json_module.JSONDecodeError:
                    pass

                travel_plan = TravelPlan.model_validate_json(json_text)
                print(f"[SUCCESS] Generated plan on attempt {attempt + 1}")
                return travel_plan

            except Exception as e:
                print(f"[WARNING] Attempt {attempt + 1} failed: {str(e)[:200]}")
                if attempt == max_retries - 1:
                    print("Error validating JSON:")
                    print(json_text if 'json_text' in locals() else response[:500])
                    raise
                print(f"Retrying... ({max_retries - attempt - 1} remaining)")

itinerary_agent = ItineraryAgent(client=client, model=MODEL)


╔══════════════════════════════════════════[ ItineraryAgent - System Prompt ]══════════════════════════════════════════╗
║ You are an expert Itinerary Planning Agent with 10+ years of experience in travel planning and optimization.         ║
║ ## Task Objective                                                                                                    ║
║ Generate a COMPLETE, VALID day-by-day travel itinerary that STRICTLY adheres to ALL constraints below.               ║
║ ## Mandatory Constraints (Must ALL be satisfied - ZERO tolerance for violations)                                     ║
║ 1. **Date Range**: Itinerary MUST span EXACTLY from arrival to departure dates (inclusive)                           ║
║ 2. **Budget**: Total cost MUST be ≤ budget (NOT exceed even by 1 unit)                                               ║
║ 3. **Activity Existence**: ONLY use activities from "Available Activities" list (NO hallucinations)                  ║
║ 4. **Activity Dates**: ONLY u

In [26]:
# Generate the travel itinerary
# No changes needed here, though you can change the model to a different one if you want.

travel_plan_1 = itinerary_agent.get_itinerary(
    vacation_info=vacation_info,
    model=MODEL,  # Optionally, you can change the model here
)

if travel_plan_1 is not None:
    print("[OK] Initial itinerary generated successfully. Congratulations!")


╔═══════════════════════════════════════════[ ItineraryAgent - User Prompt ]═══════════════════════════════════════════╗
║ {                                                                                                                    ║
║   "travelers": [                                                                                                     ║
║     {                                                                                                                ║
║       "name": "Yuri",                                                                                                ║
║       "age": 30,                                                                                                     ║
║       "interests": [                                                                                                 ║
║         "tennis",                                                                                                    ║
║         "cooking",           

## Evaluating the Itinerary

We've successfully created an itinerary, but how do we know if it's any good?

Now we will create some evaluation functions (sometimes called evals) to help us determine the quality of the itinerary. We will not only want our final output to be of the highest quality possible initially, but we also want to give the chance for the LLM to reflect on its own output and make improvements at a second pass.

If the itinerary does not meet all the criteria specified here, no worries! Afterwards, we will implement a feedback loop that will allow the agent to improve its plan iteratively.

In [27]:
# Helper functions for running the evaluation functions
# No change needed here.

class AgentError(Exception):
    pass


class EvaluationResults(BaseModel):
    success: bool
    failures: List[str]
    eval_functions: List[str]


def get_eval_results(vacation_info, final_output, eval_functions) -> EvaluationResults:
    """
    Evaluates the final output of the itinerary agent against a set of evaluation functions.
    Args:
        vacation_info (VacationInfo): The vacation information used to generate the itinerary.
        final_output (TravelPlan): The final output from the itinerary agent.
        eval_functions (List[callable]): A list of evaluation functions to apply.
    Returns:
        EvaluationResults: An object containing the success status, any failures, and the names of the evaluation functions used.
    """
    from project_lib import print_in_box
    if not isinstance(vacation_info, VacationInfo):
        raise ValueError("vacation_info must be an instance of VacationInfo")
    if not isinstance(final_output, TravelPlan):
        raise ValueError("final_output must be an instance of TravelPlan")
    if not isinstance(eval_functions, list) or not all(
        callable(fn) for fn in eval_functions
    ):
        raise ValueError("eval_functions must be a list of callable functions")
    eval_results = []
    for eval_fn in eval_functions:
        try:
            eval_fn(vacation_info, final_output)
        except AgentError as e:
            error_msg = str(e)
            print_in_box(error_msg, title="Evaluation Error")
            print("\n\n")

            eval_results.append(error_msg)
    return EvaluationResults(
        success=len(eval_results) == 0,
        failures=eval_results,
        eval_functions=[fn.__name__ for fn in eval_functions],
    )


In [28]:
# Basic evaluation functions
# No changes needed here.

def eval_start_end_dates_match(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the arrival and departure dates in vacation_info match the start and end dates in final_output.

    Args:
        vacation_info (dict): Contains the vacation details including arrival and departure dates
        final_output (dict): Contains the itinerary details including start and end dates

    Raises:
        AgentError: If either the arrival date doesn't match the start date or the departure date doesn't match the end date
    """
    if (
        vacation_info.date_of_arrival != final_output.start_date
        or vacation_info.date_of_departure != final_output.end_date
    ):
        raise AgentError(
            f"Dates do not match: {vacation_info.date_of_arrival} != {final_output.start_date} or {vacation_info.date_of_departure} != {final_output.end_date}"
        )

    if final_output.start_date > final_output.end_date:
        raise AgentError(
            f"Start date is after end date: {final_output.start_date} > {final_output.end_date}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_start_end_dates_match],
)

EvaluationResults(success=True, failures=[], eval_functions=['eval_start_end_dates_match'])

In [29]:
# Evaluation functions related to the budget and total cost
# No changes needed here.


def eval_total_cost_is_accurate(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the total cost stated in final_output matches the sum of all activity prices.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including activities with prices and total cost

    Raises:
        AgentError: If the calculated total cost doesn't match the stated total cost
    """
    actual_total_cost = 0

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            actual_total_cost += activity_recommendation.activity.price

    stated_total_cost = int(final_output.total_cost)

    if actual_total_cost != stated_total_cost:
        raise AgentError(
            f"Stated total cost does not match calculated total cost: {actual_total_cost} != {stated_total_cost}"
        )
    
def eval_total_cost_is_within_budget(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the total cost stated in final_output is within the budget specified in vacation_info.

    Args:
        vacation_info (dict): Contains the vacation details including budget
        final_output (dict): Contains the itinerary details including total cost

    Raises:
        AgentError: If the total cost exceeds the budget
    """
    stated_total_cost = int(final_output.total_cost)
    if stated_total_cost > vacation_info.budget:
        raise AgentError(
            f"Total cost exceeds budget: {stated_total_cost} > {vacation_info.budget}"
        )

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_total_cost_is_accurate, eval_total_cost_is_within_budget],
)


EvaluationResults(success=True, failures=[], eval_functions=['eval_total_cost_is_accurate', 'eval_total_cost_is_within_budget'])

In [30]:
# The final output contains copies of the activities, so we need to verify that the copies are accurate
# and not hallucinated.
# No changes needed here.

def eval_itinerary_events_match_actual_events(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that the events listed in the itinerary match the actual events

    Args:
        vacation_info (dict): Contains the vacation details including traveler information and their interests
        final_output (dict): Contains the itinerary details including daily activities

    Raises:
        AgentError: If any traveler has no matching activities or if one traveler has more than twice
                   the number of matching activities compared to another traveler
    """
    from project_lib import call_activity_by_id_api_mocked
    event_ids_not_matching = []
    event_ids_missing = []

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            event_id = activity_recommendation.activity.activity_id
            # Assuming get_event_by_id is a function that retrieves the event by its ID

            reference_event = call_activity_by_id_api_mocked(event_id)

            if reference_event is None:
                event_ids_missing.append(event_id)

            elif Activity(**reference_event) != activity_recommendation.activity:
                print(
                    "---\n"
                    f"Event ID {event_id} does not match the reference event:\n"
                    f"Reference Event: {reference_event}\n"
                    f"Activity Event: {activity_recommendation.activity.model_dump()}"
                )
                event_ids_not_matching.append(event_id)
            else:
                # The event matches, so we can continue
                pass

    if event_ids_missing or event_ids_not_matching:
        raise AgentError(
            f"Event IDs missing: {event_ids_missing}\nEvent IDs not matching: {event_ids_not_matching}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_events_match_actual_events],
)


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_events_match_actual_events'])

In [31]:
# Check that the itinerary includes at least one activity matching each traveler's interests.
# No changes needed here.

def eval_itinerary_satisfies_interests(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that the itinerary includes activities matching each traveler's interests.

    This function checks that each traveler has at least one activity in the itinerary that matches their interests.

        Args:
        vacation_info (dict): Contains the vacation details including traveler information and their interests
        final_output (dict): Contains the itinerary details including daily activities

    Raises:
        AgentError: If any traveler has no matching activities or if one traveler has more than twice
                   the number of matching activities compared to another traveler
    """
    traveler_to_interests = {}
    traveler_to_interest_hit_counts = {}

    for traveler in vacation_info.travelers:
        traveler_to_interests[traveler.name] = traveler.interests
        traveler_to_interest_hit_counts[traveler.name] = 0

    for traveler_name, interests in traveler_to_interests.items():
        for itinerary_day in final_output.itinerary_days:
            for activity_recommendation in itinerary_day.activity_recommendations:
                # Check if the activity matches any of the traveler's interests
                matching_interests = set(traveler_to_interests[traveler_name]) & set(
                    activity_recommendation.activity.related_interests
                )

                if matching_interests:
                    traveler_to_interest_hit_counts[traveler_name] += 1
                    print(
                        f"[OK] Traveler {traveler_name} has a match with interest {matching_interests} at {activity_recommendation.activity.name}"
                    )

    travelers_with_no_interest_hits = [
        traveler
        for traveler, interest_hit_count in traveler_to_interest_hit_counts.items()
        if interest_hit_count == 0
    ]

    # If any of the travelers have 0 matches, raise an error
    if travelers_with_no_interest_hits:
        raise AgentError(
            f"Travelers {travelers_with_no_interest_hits} has no matches with the itinerary."
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_satisfies_interests],
)


[OK] Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
[OK] Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
[OK] Traveler Hiro has a match with interest {reading, art} at AgentsVille Twilight Writing Escape
[OK] Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Hiro has a match with interest {music} at Morning Groove Dance Party


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_satisfies_interests'])

In [32]:
# Use an LLM to determine whether an activity should be avoided due to weather
# COMPLETED: Prompt to evaluate compatibility between activity and weather

# EXPLANATION: This prompt uses an LLM to determine if an activity is appropriate
# given the weather. For example, an outdoor activity without indoor backup
# should not be scheduled on a rainy day.

ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = """
You are an expert travel advisor evaluating whether activities are suitable for given weather conditions.

## Task

Determine if an activity is COMPATIBLE or INCOMPATIBLE with the weather condition.

### Guidelines:

1. **INCOMPATIBLE scenarios**:
   - Outdoor-only activities (hiking, beach, outdoor sports) during RAIN
   - Activities explicitly requiring good weather during bad conditions

2. **COMPATIBLE scenarios**:
   - Indoor activities (museums, theaters, cooking classes) in ANY weather
   - Activities with indoor backup options mentioned in description
   - Outdoor activities on sunny or cloudy (but not rainy) days
   - When there's NOT enough information about weather sensitivity, assume IS_COMPATIBLE

3. **Special cases**:
   - If activity description mentions "indoor alternative" or "rain plan", it's COMPATIBLE
   - Light rain vs heavy rain: Use judgment based on activity description

**IMPORTANT**: When in doubt or when information is insufficient, default to IS_COMPATIBLE.

## Output format

    REASONING:
    * Weather condition: [state the weather]
    * Activity type: [indoor/outdoor/mixed]
    * Weather sensitivity: [high/medium/low]
    * Key factors: [list relevant factors from description]
    * Decision rationale: [explain your reasoning]

    FINAL ANSWER:
    [IS_COMPATIBLE or IS_INCOMPATIBLE]

## Examples

Example 1:
Activity: Outdoor Hiking Trail
Description: A scenic 5-mile hiking trail through the mountains
Weather Condition: Rain

REASONING:
* Weather condition: Rain
* Activity type: Outdoor only
* Weather sensitivity: High
* Key factors: Hiking trails become slippery and dangerous in rain
* Decision rationale: This is purely outdoor with no indoor backup

FINAL ANSWER:
IS_INCOMPATIBLE

---

Example 2:
Activity: City Museum Tour
Description: Explore art and history exhibits in our indoor museum
Weather Condition: Rain

REASONING:
* Weather condition: Rain
* Activity type: Indoor
* Weather sensitivity: None
* Key factors: Completely indoor activity
* Decision rationale: Museums are perfect for rainy days

FINAL ANSWER:
IS_COMPATIBLE

---

Example 3:
Activity: Farmers Market
Description: Browse local vendors. Indoor pavilion available in case of rain.
Weather Condition: Rain

REASONING:
* Weather condition: Rain
* Activity type: Mixed with indoor backup
* Weather sensitivity: Low
* Key factors: Description explicitly mentions indoor pavilion for rain
* Decision rationale: Has rain contingency plan

FINAL ANSWER:
IS_COMPATIBLE
""".strip()



def eval_activities_and_weather_are_compatible(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that no outdoor-only activities are scheduled during inclement weather."""
    from project_lib import do_chat_completion

    activities_that_are_incompatible = []

    for itinerary_day in final_output.itinerary_days:
        weather_condition = itinerary_day.weather.condition

        for activity_recommendation in itinerary_day.activity_recommendations:
            resp = do_chat_completion(
                messages=[
                    {
                        "role": "system",
                        "content": ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT,
                    },
                    {
                        "role": "user",
                        "content": f"Activity: {activity_recommendation.activity.name}\nDescription: {activity_recommendation.activity.description}\nWeather Condition: {weather_condition}",
                    },
                ],
                client=client,
                model=OpenAIModel.GPT_41_NANO,
            )

            if "IS_COMPATIBLE" in (resp or ""):
                is_compatible = True
            elif "IS_INCOMPATIBLE" in (resp or ""):
                is_compatible = False
            else:
                raise RuntimeError(
                    f"Unexpected response: {resp}"
                )

            if is_compatible:
                print(f"[OK] Activity {activity_recommendation.activity.name} compatible with {weather_condition}")
            else:
                activities_that_are_incompatible.append(activity_recommendation.activity.name)
                print(f"[ERROR] Activity {activity_recommendation.activity.name} incompatible with {weather_condition}")

    if activities_that_are_incompatible:
        raise AgentError(f"Activities incompatible with weather: {activities_that_are_incompatible}")


eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_activities_and_weather_are_compatible],
)

eval_results

[OK] Activity Serve & Savor: Tennis and Taste Luncheon compatible with clear
[OK] Activity AgentsVille Twilight Writing Escape compatible with clear
[OK] Activity Palette & Palate: Art Meets Cooking Experience compatible with partly cloudy
[OK] Activity Morning Groove Dance Party compatible with partly cloudy
[ERROR] Activity AgentsVille Nature & Green Thumb Adventure incompatible with thunderstorm
[OK] Activity Tech & Film Fusion Night compatible with thunderstorm

╔═════════════════════════════════════════════════[ Evaluation Error ]═════════════════════════════════════════════════╗
║ Activities incompatible with weather: ['AgentsVille Nature & Green Thumb Adventure']                                 ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝





EvaluationResults(success=False, failures=["Activities incompatible with weather: ['AgentsVille Nature & Green Thumb Adventure']"], eval_functions=['eval_activities_and_weather_are_compatible'])

In [33]:
# Run all of the evaluation functions again
# No changes needed here.

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
]

eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

eval_results.model_dump()

[OK] Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
[OK] Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
[OK] Traveler Hiro has a match with interest {reading, art} at AgentsVille Twilight Writing Escape
[OK] Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Hiro has a match with interest {music} at Morning Groove Dance Party
[OK] Activity Serve & Savor: Tennis and Taste Luncheon compatible with clear
[OK] Activity AgentsVille Twilight Writing Escape compatible with clear
[OK] Activity Palette & Palate: Art Meets Cooking Experience compatible with partly cloudy
[OK] Activity Morning Groove Dance Party compatible with partly cloudy
[ERROR] Activity AgentsVille Nature & Green Thumb Adventure incompatible with thunderstorm
[OK] Activity Tec

{'success': False,
 'failures': ["Activities incompatible with weather: ['AgentsVille Nature & Green Thumb Adventure']"],
 'eval_functions': ['eval_start_end_dates_match',
  'eval_total_cost_is_accurate',
  'eval_itinerary_events_match_actual_events',
  'eval_itinerary_satisfies_interests',
  'eval_total_cost_is_within_budget',
  'eval_activities_and_weather_are_compatible']}

## Defining the Tools

Our ItineraryRevisionAgent will be a ReAct-based agent that will use tools to:
- Evaluate/Re-evaluate the itinerary
- Use a calculator since LLMs sometimes struggle with arithmetic
- Call the activities API to get more information about activities
- Return the final itinerary


In [34]:
# Helper function to generate tool descriptions from function docstrings
# No changes needed here.

def get_tool_descriptions_string(fns):
    """Generates a tool description from a function's docstring.
    Args:
        fns (list): List of functions to generate descriptions for.
    Returns:
        str: A formatted string containing the function names and their descriptions."""
    resp = ""
    for fn in fns:
        function_name = fn.__name__
        function_doc = fn.__doc__ or "No description provided."

        resp += f"* `{function_name}`: {function_doc}\n"

    return resp

In [35]:
# Define the calculator tool that evaluates mathematical expressions.
# No changes needed here.

def calculator_tool(input_expression) -> float:
    """Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    """
    import numexpr as ne
    return float(ne.evaluate(input_expression))


assert calculator_tool("1 + 1") == 2.0

print(get_tool_descriptions_string([calculator_tool]))

* `calculator_tool`: Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    



In [36]:
# Tool to fetch activities for a given date and city
# COMPLETED: Complete docstring for the activities tool

# EXPLANATION: In ReAct, tools need very clear docstrings
# explaining exactly what parameters they accept and what they return.
# The LLM uses this information to decide how to call the tool.

def get_activities_by_date_tool(date: str, city: str) -> List[dict]:
    """Fetches all available activities for a specific date and city.

    This tool retrieves the complete list of activities available on a given date
    in a specified city. Use this when you need to know what activities are available
    on a particular day to plan the itinerary.

    Args:
        date (str): The date in YYYY-MM-DD format (e.g., "2025-06-10")
        city (str): The name of the city (e.g., "AgentsVille")

    Returns:
        List[dict]: A list of dictionaries, each representing an available activity
                    with fields: activity_id, name, start_time, end_time, location,
                    description, price, and related_interests.

    Example:
        >>> activities = get_activities_by_date_tool("2025-06-10", "AgentsVille")
        >>> print(len(activities))  # Number of activities available that day
    """
    from project_lib import call_activities_api_mocked
    resp = call_activities_api_mocked(date=date, city=city)
    return [Activity.model_validate(activity).model_dump() for activity in resp]

assert len(get_activities_by_date_tool("2025-06-10", "AgentsVille")) > 0

print(get_tool_descriptions_string([get_activities_by_date_tool]))

* `get_activities_by_date_tool`: Fetches all available activities for a specific date and city.

    This tool retrieves the complete list of activities available on a given date
    in a specified city. Use this when you need to know what activities are available
    on a particular day to plan the itinerary.

    Args:
        date (str): The date in YYYY-MM-DD format (e.g., "2025-06-10")
        city (str): The name of the city (e.g., "AgentsVille")

    Returns:
        List[dict]: A list of dictionaries, each representing an available activity
                    with fields: activity_id, name, start_time, end_time, location,
                    description, price, and related_interests.

    Example:
        >>> activities = get_activities_by_date_tool("2025-06-10", "AgentsVille")
        >>> print(len(activities))  # Number of activities available that day
    



In [37]:
# Tool to run all evaluation functions on a travel plan.
# No changes needed here.

def run_evals_tool(travel_plan: TravelPlan) -> dict:
    """Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    """
    if isinstance(travel_plan, dict):
        travel_plan = TravelPlan.model_validate(travel_plan)

    resp = get_eval_results(
        vacation_info=vacation_info,
        final_output=travel_plan,
        eval_functions=ALL_EVAL_FUNCTIONS,
    )
    return {
        # Show the success status and any failures
        "success": resp.success,
        "failures": resp.failures,
    }


print(get_tool_descriptions_string([run_evals_tool]))

* `run_evals_tool`: Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    



In [38]:
# Let's double check that the tool works as expected.
# You should see the same results as before
run_evals_tool(travel_plan=travel_plan_1)

[OK] Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
[OK] Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
[OK] Traveler Hiro has a match with interest {reading, art} at AgentsVille Twilight Writing Escape
[OK] Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Hiro has a match with interest {music} at Morning Groove Dance Party
[OK] Activity Serve & Savor: Tennis and Taste Luncheon compatible with clear
[OK] Activity AgentsVille Twilight Writing Escape compatible with clear
[OK] Activity Palette & Palate: Art Meets Cooking Experience compatible with partly cloudy
[OK] Activity Morning Groove Dance Party compatible with partly cloudy
[ERROR] Activity AgentsVille Nature & Green Thumb Adventure incompatible with thunderstorm
[OK] Activity Tec

{'success': False,
 'failures': ["Activities incompatible with weather: ['AgentsVille Nature & Green Thumb Adventure']"]}

In [39]:
# A tool to return the final travel plan
# No changes needed here.

def final_answer_tool(final_output: TravelPlan) -> TravelPlan:
    """Returns the final travel plan

    Args:
        final_output (TravelPlan): The final travel plan to return.

    Returns:
        TravelPlan: The final travel plan.
    """
    return final_output


print(get_tool_descriptions_string([final_answer_tool]))

* `final_answer_tool`: Returns the final travel plan

    Args:
        final_output (TravelPlan): The final travel plan to return.

    Returns:
        TravelPlan: The final travel plan.
    



In [40]:
# List of all tools available for the agent
# No changes needed here.

ALL_TOOLS = [
    calculator_tool,
    get_activities_by_date_tool,
    run_evals_tool,
    final_answer_tool,
]
print(get_tool_descriptions_string(ALL_TOOLS))

* `calculator_tool`: Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    
* `get_activities_by_date_tool`: Fetches all available activities for a specific date and city.

    This tool retrieves the complete list of activities available on a given date
    in a specified city. Use this when you need to know what activities are available
    on a particular day to plan the itinerary.

    Args:
        date (str): The date in YYYY-MM-DD format (e.g., "2025-06-10")
        city (str): The name of the city (e.g., "AgentsVille")

    Returns:
        List[dict]: A list of dictionaries, each representing an available activity
                    with fields: activity_id, name, start_time, end_time, location,
                    desc

## The ItineraryRevisionAgent

The ItineraryRevisionAgent will
* take initial feedback from the user about the itinerary and
* use the tools defined above

to refine original itinerary iteratively using a ReAct-based approach.

In [41]:
# Get the traveler's feedback and create a new evaluation function to check if the feedback was incorporated.
# No changes needed here.

TRAVELER_FEEDBACK = "I want to have at least two activities per day."


def eval_traveler_feedback_is_incorporated(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Checks if the traveler's feedback was incorporated into the revised travel plan.

    Args:
        vacation_info (VacationInfo): The vacation information.
        final_output (TravelPlan): The revised travel plan.

    Raises:
        AgentError: If the traveler's feedback was not successfully incorporated.
    """

    agent = ChatAgent(
        system_prompt="""You are an expert in evaluating whether a travel plan incorporates traveler feedback.

    ## Output Format

    Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

        ANALYSIS:
        * [step-by-step analysis]


        FINAL OUTPUT:
        [FULLY_INCORPORATED, PARTIALLY_INCORPORATED, NOT_INCORPORATED, or UNKNOWN]
        REASON: [reasoning for the final output]

    """,
        client=client,
        model=OpenAIModel.GPT_41,  # Use a powerful model for checking traveler feedback
    )

    resp = agent.chat(
        f"""Traveler Feedback: {TRAVELER_FEEDBACK}
    Revised Travel Plan: {final_output.model_dump_json()}
    """,
    )
    if "FINAL OUTPUT:" not in resp:
        raise RuntimeError(
            f"Unexpected response from the model: {resp}. Expected 'FINAL OUTPUT:'."
        )
    if "FULLY_INCORPORATED" not in resp:
        final_output = resp.split("FINAL OUTPUT:")[-1].strip()
        raise AgentError(
            f"Traveler feedback was not successfully incorporated into the revised travel plan. Response: {final_output}"
        )

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
    eval_traveler_feedback_is_incorporated,  # Add this new evaluation
]

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

[OK] Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
[OK] Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
[OK] Traveler Hiro has a match with interest {reading, art} at AgentsVille Twilight Writing Escape
[OK] Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Hiro has a match with interest {music} at Morning Groove Dance Party
[OK] Activity Serve & Savor: Tennis and Taste Luncheon compatible with clear
[OK] Activity AgentsVille Twilight Writing Escape compatible with clear
[OK] Activity Palette & Palate: Art Meets Cooking Experience compatible with partly cloudy
[OK] Activity Morning Groove Dance Party compatible with partly cloudy
[ERROR] Activity AgentsVille Nature & Green Thumb Adventure incompatible with thunderstorm
[OK] Activity Tec

EvaluationResults(success=False, failures=["Activities incompatible with weather: ['AgentsVille Nature & Green Thumb Adventure']"], eval_functions=['eval_start_end_dates_match', 'eval_total_cost_is_accurate', 'eval_itinerary_events_match_actual_events', 'eval_itinerary_satisfies_interests', 'eval_total_cost_is_within_budget', 'eval_activities_and_weather_are_compatible', 'eval_traveler_feedback_is_incorporated'])

In [42]:
# ENHANCED: Production-level ReAct prompt - MLOps/AI Engineer Version
# Original prompt critiques:
# - Does not clearly specify when to use each tool
# - Missing recovery strategy for tool failures
# - No clear iteration limit per error type
# - Could enter infinite loops without detection
# IMPROVEMENTS: Tool selection strategy, error recovery, loop detection, determinism

from project_lib import print_in_box
# [OK] MEJORAS: Tool selection strategy, error recovery, loop detection, determinismo

from project_lib import print_in_box

# MEJORAS CRÍTICAS APLICADAS:
# 1. Tool selection decision tree clara
# 2. Error recovery strategies por tipo de fallo
# 3. Loop detection (no llamar mismo tool con mismos args 2+ veces)
# 4. Max iterations explícito por fase
# 5. Temperatura 0.1 para máximo determinismo en decisions
# 6. Metrics tracking (costs, time, iterations)

ITINERARY_REVISION_AGENT_SYSTEM_PROMPT = f"""You are a Production-Grade Itinerary Revision Agent (v2.0) with advanced error recovery and optimization capabilities.

## Mission Critical Objective

Revise the provided travel itinerary to achieve `success: true` on ALL evaluation criteria in MINIMUM iterations.

## Operational Phases (Follow in STRICT ORDER)

### Phase 1: DIAGNOSIS (Iterations 1-2)
**Goal**: Understand ALL problems before attempting fixes

**Required Actions**:
1. Call `run_evals_tool` with current plan → Get comprehensive failure list
2. Categorize failures by type:
   - Type A: Missing/incorrect data (dates, activities)
   - Type B: Constraint violations (budget, weather)
   - Type C: Feedback not incorporated (traveler requests)

**Output**: Failure inventory with root causes

### Phase 2: DATA GATHERING (Iterations 3-5)
**Goal**: Collect information needed for fixes

**Tool Selection Decision Tree**:
```
IF (weather incompatibility detected):
    → get_activities_by_date_tool for affected dates
    → Filter for indoor activities

IF (budget exceeded):
    → calculator_tool to verify math
    → Identify most expensive activities
    → Find cheaper alternatives via get_activities_by_date_tool

IF (interest not matched):
    → Review available activities via get_activities_by_date_tool
    → Filter by interest categories

IF (feedback: "X activities per day"):
    → Calculate current activities per day
    → Identify days below threshold
    → get_activities_by_date_tool for those days
```

### Phase 3: FIX APPLICATION (Iterations 6-12)
**Goal**: Apply fixes based on gathered data

**Fix Priority Order**:
1. P0 (Blocker): Date mismatches, non-existent activities
2. P1 (Critical): Budget violations, weather incompatibility
3. P2 (High): Interest coverage, feedback incorporation
4. P3 (Medium): Activity distribution, time optimization

**Application Strategy**:
- Fix one category at a time (don't mix)
- Verify each fix doesn't break other constraints
- Use calculator_tool for ANY arithmetic (costs, counts)

### Phase 4: VERIFICATION (Iteration 13-14)
**Goal**: Confirm ALL criteria pass before final submission

**Required Actions**:
1. Call `run_evals_tool` on revised plan
2. IF `success: false` → Return to Phase 3 (max 2 additional cycles)
3. IF `success: true` → Proceed to Phase 5

### Phase 5: FINALIZATION (Iteration 15)
**Goal**: Submit validated solution

**Required Action**:
- Call `final_answer_tool` with complete, validated TravelPlan

## Tool Catalog & Usage Guidelines

### 1. calculator_tool
**When to use**:
- ANY cost calculation (totals, running sums, per-day averages)
- Counting activities per day
- Budget remaining calculations

**Format**: `{{"tool_name": "calculator_tool", "arguments": {{"input_expression": "10 + 20 + 30"}}}}`

**Anti-patterns (DON'T)**:
- Manual arithmetic in THOUGHT (always use tool)
- Complex nested expressions (break into steps)

### 2. get_activities_by_date_tool
**When to use**:
- Need to see ALL activities for a specific date
- Searching for alternative activities
- Verifying activity availability

**Format**: `{{"tool_name": "get_activities_by_date_tool", "arguments": {{"date": "2025-06-10", "city": "AgentsVille"}}}}`

**Anti-patterns (DON'T)**:
- Calling for dates outside trip range
- Assuming activity availability without checking

### 3. run_evals_tool
**When to use**:
- Beginning of cycle (Phase 1)
- After applying fixes (Phase 4)
- MANDATORY before final_answer_tool

**Format**: `{{"tool_name": "run_evals_tool", "arguments": {{"travel_plan": {{...complete_plan...}}}}}}`

**Anti-patterns (DON'T)**:
- Calling with partial/incomplete plans
- Skipping before final answer

### 4. final_answer_tool
**When to use**:
- ONLY after run_evals_tool returns `success: true`
- ALL feedback incorporated
- Phase 5 reached

**Format**: `{{"tool_name": "final_answer_tool", "arguments": {{"final_output": {{...complete_valid_plan...}}}}}}`

**Anti-patterns (DON'T)**:
- Calling before final validation
- Submitting plan that failed any evaluation

## ReAct Cycle Format (MANDATORY STRUCTURE)

Every response MUST contain EXACTLY:

```
THOUGHT:
[Structured reasoning following this template:]

1. Current Status:
   - Phase: [1-5]
   - Iteration: [X/15]
   - Failures remaining: [X]
   - Tools used so far: [list]

2. Analysis:
   - What did last OBSERVATION tell me?
   - What's the next logical step?
   - Which tool solves this specific problem?

3. Decision:
   - Tool to use: [name]
   - Why this tool: [justification]
   - Expected outcome: [what I'll learn/fix]
   - Parameters needed: [list with values]

4. Loop Detection Check:
   - Have I called this tool with these exact args before? [yes/no]
   - If yes: What will I do differently this time?

ACTION:
{{"tool_name": "[exact_tool_name]", "arguments": {{"param1": "value1", "param2": "value2"}}}}
```

## Error Recovery Strategies

**Error Type 1: Invalid JSON in ACTION**
→ Review JSON syntax rules
→ Ensure all strings use double quotes
→ Check for missing commas, braces
→ Retry with corrected JSON

**Error Type 2: Tool execution error**
→ Review tool docstring for correct parameters
→ Verify parameter types (string vs int vs dict)
→ Check parameter values are valid (dates in range, etc.)
→ Retry with corrected parameters

**Error Type 3: Evaluation still failing after fixes**
→ Re-read the EXACT failure message
→ Verify fix was actually applied to plan
→ Check if fix broke another constraint
→ Try alternative approach

**Error Type 4: Approaching iteration limit (12+)**
→ Identify which failure is hardest to fix
→ Focus ONLY on that one issue
→ Use simpler solution (remove rather than replace)
→ Prioritize getting to `success: true` over perfect plan

## Loop Detection & Prevention

**System will auto-detect**:
- Same tool called 3+ times with identical arguments
- No progress on failure count after 3 iterations
- Cost/date/count unchanged after "fix" applied

**Your responsibility**:
- Track in THOUGHT what you tried before
- If stuck, try DIFFERENT approach (different tool or different parameters)
- If truly stuck after 3 attempts, simplify solution (remove problematic activities)

## Mandatory Traveler Feedback

**Current Feedback**: "{TRAVELER_FEEDBACK}"

**Implementation Requirements**:
- Parse feedback to extract specific requirement (e.g., "at least 2 activities per day")
- Create verifiable criterion (count activities per day, ensure all ≥ 2)
- Add activities if needed using get_activities_by_date_tool
- MUST pass eval_traveler_feedback_is_incorporated before finalizing

## Success Criteria (ALL must be TRUE)

1. [OK] `run_evals_tool` returns `success: true`
2. [OK] `failures: []` (empty list)
3. [OK] Traveler feedback incorporated
4. [OK] All 7 mandatory constraints satisfied (see ItineraryAgent prompt)
5. [OK] Used ≤ 15 iterations
6. [OK] No tool called 3+ times with identical args (loop detection)

## Available Tools Reference

{get_tool_descriptions_string(ALL_TOOLS)}

## Critical Context

### TravelPlan JSON Schema:
{json.dumps(TravelPlan.model_json_schema(), indent=2)}

### Original VacationInfo:
{vacation_info.model_dump_json(indent=2)}

### Activity Schema:
{json.dumps(Activity.model_json_schema(), indent=2)}

### Weather Schema:
{json.dumps(Weather.model_json_schema(), indent=2)}

## Concrete Examples (Learn from these)

**Example 1 - Phase 1 (Initial Diagnosis)**:

THOUGHT:
1. Current Status: Phase: 1, Iteration: 1/15, Failures: Unknown, Tools used: None
2. Analysis: Starting revision cycle. Need to understand what's wrong with current plan.
3. Decision: Tool to use: run_evals_tool. Why: Get comprehensive failure list. Expected: List of specific failures with categories.
4. Loop Detection: First call, no loop risk.

ACTION:
{{"tool_name": "run_evals_tool", "arguments": {{"travel_plan": {{"city": "AgentsVille", "start_date": "2025-06-10", ...}}}}}}

---

**Example 2 - Phase 2 (Data Gathering after finding weather issue)**:

THOUGHT:
1. Current Status: Phase: 2, Iteration: 4/15, Failures: 2 (weather incompatibility, feedback), Tools used: [run_evals_tool x2]
2. Analysis: OBSERVATION showed outdoor activity on rainy June 11. Need indoor alternatives.
3. Decision: Tool: get_activities_by_date_tool. Why: Find indoor options for June 11. Expected: List of activities, filter for indoor.
4. Loop Detection: First time calling for June 11, no loop risk.

ACTION:
{{"tool_name": "get_activities_by_date_tool", "arguments": {{"date": "2025-06-11", "city": "AgentsVille"}}}}

---

**Example 3 - Phase 3 (Applying fix with calculator)**:

THOUGHT:
1. Current Status: Phase: 3, Iteration: 8/15, Failures: 1 (budget exceeded by 15), Tools used: [run_evals_tool x2, get_activities x3, calculator x1]
2. Analysis: Current total is 145, budget is 130. Need to verify calc and identify expensive activity to replace.
3. Decision: Tool: calculator_tool. Why: Verify sum: 30+25+40+25+25. Expected: 145 confirmed, then find 40-cost activity to replace.
4. Loop Detection: Used calculator once before for different calc, no loop.

ACTION:
{{"tool_name": "calculator_tool", "arguments": {{"input_expression": "30 + 25 + 40 + 25 + 25"}}}}

---

**Example 4 - Phase 4 (Final verification)**:

THOUGHT:
1. Current Status: Phase: 4, Iteration: 13/15, Failures: 0 (after fixes), Tools used: [run_evals_tool x3, get_activities x4, calculator x3]
2. Analysis: Applied all fixes. Need to verify EVERYTHING passes before finalizing.
3. Decision: Tool: run_evals_tool. Why: Mandatory pre-submission validation. Expected: success: true.
4. Loop Detection: Third run_evals call, but different plan each time (progress made), no infinite loop.

ACTION:
{{"tool_name": "run_evals_tool", "arguments": {{"travel_plan": {{"city": "AgentsVille", ...improved_plan...}}}}}}

---

**Example 5 - Phase 5 (Finalization)**:

THOUGHT:
1. Current Status: Phase: 5, Iteration: 14/15, Failures: 0, Tools used: [run_evals_tool x4, get_activities x4, calculator x3]
2. Analysis: OBSERVATION confirmed success: true. All criteria met. Feedback incorporated. Ready to submit.
3. Decision: Tool: final_answer_tool. Why: Submit validated solution. Expected: Cycle complete.
4. Loop Detection: N/A - final action.

ACTION:
{{"tool_name": "final_answer_tool", "arguments": {{"final_output": {{"city": "AgentsVille", "start_date": "2025-06-10", "end_date": "2025-06-12", "total_cost": 125, "itinerary_days": [...]}}}}}}

## MLOps Best Practices Applied

- **Determinism**: temperature=0.1, structured reasoning format
- **Observability**: Iteration tracking, tool usage logging, failure categorization
- **Error Handling**: Retry logic, graceful degradation, fallback strategies
- **Performance**: Max 15 iterations, early termination on success
- **Validation**: Pre-submission checks, schema validation, constraint verification
- **Debugging**: Loop detection, tool call history, phase tracking

"""  # noqa


class ItineraryRevisionAgent(ChatAgent):
    system_prompt = ITINERARY_REVISION_AGENT_SYSTEM_PROMPT
    tools = ALL_TOOLS

    def get_observation_string(self, tool_call_obj) -> str:
        """Extracts the observation from the thought-action response with enhanced error messages."""

        if "tool_name" not in tool_call_obj:
            return "OBSERVATION ERROR: No 'tool_name' key found in ACTION JSON. Required format: {\"tool_name\": \"tool_name_here\", \"arguments\": {...}}"

        if "arguments" not in tool_call_obj:
            return "OBSERVATION ERROR: No 'arguments' key found in ACTION JSON. Required format: {\"tool_name\": \"...\", \"arguments\": {\"param\": \"value\"}}"

        if not isinstance(tool_call_obj["arguments"], dict):
            return f"OBSERVATION ERROR: 'arguments' must be a dictionary/object. Got {type(tool_call_obj['arguments']).__name__} instead. Fix: Use {{}} for arguments."

        if not isinstance(tool_call_obj["tool_name"], str):
            return f"OBSERVATION ERROR: 'tool_name' must be a string. Got {type(tool_call_obj['tool_name']).__name__}. Fix: Use quotes around tool name."

        tool_name = tool_call_obj["tool_name"]
        arguments = tool_call_obj["arguments"]

        # Find tool function
        tool_fn = None
        for tool in self.tools:
            if tool.__name__ == tool_name:
                tool_fn = tool
                break

        if tool_fn is None:
            available_tools = [t.__name__ for t in self.tools]
            return f"OBSERVATION ERROR: Unknown tool '{tool_name}'. Available tools: {available_tools}. Check spelling and use exact tool name."

        # Execute tool with error handling
        try:
            tool_response = tool_fn(**arguments)
            # Success - return observation
            if isinstance(tool_response, dict) and 'success' in tool_response:
                success_indicator = "[OK] SUCCESS" if tool_response.get('success') else "[WARNING]  PARTIAL"
                return f"OBSERVATION ({success_indicator}): Tool {tool_name} executed. Response: {json.dumps(tool_response, indent=2)}"
            else:
                return f"OBSERVATION ([OK] SUCCESS): Tool {tool_name} executed. Response: {tool_response}"
                
        except TypeError as e:
            # Parameter mismatch
            import inspect
            sig = inspect.signature(tool_fn)
            expected_params = list(sig.parameters.keys())
            return f"OBSERVATION ERROR: Parameter mismatch for {tool_name}. Expected parameters: {expected_params}. You provided: {list(arguments.keys())}. Error: {str(e)}"
            
        except Exception as e:
            # Other errors
            return f"OBSERVATION ERROR: Tool {tool_name} execution failed. Error type: {type(e).__name__}. Message: {str(e)}. Review tool docstring and retry with corrected parameters."

    def run_react_cycle(
        self, original_travel_plan: TravelPlan, max_steps: int = 15, model: Optional[OpenAIModel] = None, client = None,
    ) -> TravelPlan:
        """Runs the ReAct cycle with enhanced monitoring and loop detection."""
        from json_repair import repair_json
        import time
        
        start_time = time.time()
        tool_call_history = []  # Track tool calls for loop detection

        # Provide the original travel plan to revise
        self.add_message(
            role="user",
            content=f"Here is the itinerary for revision:\n{original_travel_plan.model_dump_json()}\n\nBegin Phase 1: DIAGNOSIS",
        )
        resp = None

        # Run the ReAct cycle for a maximum number of steps
        for step in range(max_steps):
            iteration_start = time.time()
            
            # Get the thought-action response from the agent
            resp = self.get_response(model=model, client=client, temperature=0.1) or ""

            # Check for THOUGHT and ACTION
            if "THOUGHT:" not in resp:
                self.add_message(role="user", content="ERROR: Missing THOUGHT section. Every response must include both THOUGHT and ACTION sections. Review prompt format.")
                continue
                
            if "ACTION:" not in resp:
                self.add_message(role="user", content="ERROR: Missing ACTION section. Every response must include ACTION with tool call JSON. Review prompt format.")
                continue

            action_string = resp.split("ACTION:")[1].strip()

            # Parse the tool call JSON from the action string
            try:
                # Fix any JSON formatting issues
                action_string = repair_json(action_string)
                tool_call_obj = json.loads(action_string)
            except json.JSONDecodeError as e:
                self.add_message(
                    role="user",
                    content=f"JSON PARSE ERROR: Could not parse ACTION JSON. Error at position {e.pos}: {e.msg}\n\nYour ACTION string:\n{action_string}\n\nFix: Check for missing quotes, commas, or braces. Use double quotes for strings.",
                )
                continue

            tool_name = tool_call_obj.get("tool_name", None)
            
            # Loop detection
            tool_signature = f"{tool_name}:{json.dumps(tool_call_obj.get('arguments', {}), sort_keys=True)}"
            if tool_signature in tool_call_history[-3:]:  # Same call in last 3 attempts
                self.add_message(
                    role="user",
                    content=f"[WARNING]  LOOP DETECTED: You've called {tool_name} with identical arguments in your last 3 iterations. This suggests you're stuck in a loop. Try a DIFFERENT approach: use a different tool, different parameters, or simplify your solution.",
                )
                continue
            tool_call_history.append(tool_signature)

            # Handle final answer tool
            if tool_name == "final_answer_tool":
                try:
                    new_travel_plan = TravelPlan.model_validate(
                        tool_call_obj["arguments"].get("final_output", tool_call_obj["arguments"])
                    )
                    elapsed = time.time() - start_time
                    print(f"\n[OK] ReAct cycle completed successfully in {step + 1} iterations ({elapsed:.1f}s)")
                    return new_travel_plan
                except Exception as e:
                    self.add_message(
                        role="user", content=f"VALIDATION ERROR: final_answer_tool received invalid TravelPlan. Pydantic validation failed: {str(e)[:500]}\n\nReview TravelPlan schema and ensure your plan matches ALL required fields and types."
                    )
                    continue

            # Execute other tools
            else:
                observation_string = self.get_observation_string(tool_call_obj=tool_call_obj)
                iteration_elapsed = time.time() - iteration_start
                observation_with_metrics = f"{observation_string}\n\n[Iteration {step + 1}/{max_steps} | Time: {iteration_elapsed:.1f}s | Tools used: {len(set(tool_call_history))} unique]"
                self.add_message(role="user", content=observation_with_metrics)

        # Max iterations reached
        elapsed = time.time() - start_time
        raise RuntimeError(
            f"[ERROR] ReAct cycle did not complete within {max_steps} iterations ({elapsed:.1f}s).\n\n"
            f"Tools called: {len(tool_call_history)} times ({len(set(tool_call_history))} unique)\n\n"
            f"Last response:\n{resp[:1000]}\n\n"
            f"Debugging tips:\n"
            f"1. Check if you're stuck in a loop (same tool/args repeatedly)\n"
            f"2. Review if evaluation failures are decreasing each iteration\n"
            f"3. Consider simplifying solution (remove activities vs finding perfect replacements)\n"
            f"4. Verify you're calling run_evals_tool before final_answer_tool"
        )

# Instantiate the Itinerary Revision Agent
itinerary_revision_agent = ItineraryRevisionAgent()

# Let's get a single THOUGHT/ACTION response back to check that the agent is working as expected.
resp = itinerary_revision_agent.chat(
    user_message=f"Here is the itinerary for revision: {travel_plan_1.model_dump_json(indent=2)}",
    add_to_messages=False,
    model=MODEL,
    client=client,
) or ""

print_in_box(resp, "Raw Response - Validation Check")

# Enhanced validation checks
validation_results = []

if "THOUGHT:" in resp:
    validation_results.append("[OK] `THOUGHT:` section present")
else:
    validation_results.append("[ERROR] Missing `THOUGHT:` section - CRITICAL ERROR")
    
if "ACTION:" in resp:
    validation_results.append("[OK] `ACTION:` section present")
else:
    validation_results.append("[ERROR] Missing `ACTION:` section - CRITICAL ERROR")
    
if "\"tool_name\"" in resp:
    validation_results.append("[OK] `tool_name` key found in ACTION")
else:
    validation_results.append("[ERROR] Missing `tool_name` in ACTION - CRITICAL ERROR")
    
if "\"arguments\"" in resp:
    validation_results.append("[OK] `arguments` key found in ACTION")
else:
    validation_results.append("[ERROR] Missing `arguments` in ACTION - CRITICAL ERROR")

# Check for structured thinking
if "Current Status:" in resp or "Phase:" in resp:
    validation_results.append("[OK] Structured THOUGHT format (includes status/phase)")
else:
    validation_results.append("[WARNING]  THOUGHT could be more structured (add status/phase)")

if "Loop Detection" in resp or "Have I called" in resp:
    validation_results.append("[OK] Loop detection check present")
else:
    validation_results.append("[WARNING]  Loop detection check missing (recommended)")

print("\n" + "="*60)
print("VALIDATION RESULTS:")
print("="*60)
for result in validation_results:
    print(result)

critical_failures = [r for r in validation_results if r.startswith("[ERROR]")]
if critical_failures:
    print("\n[WARNING]  CRITICAL: Found validation failures. Review system prompt (output format section).")
else:
    print("\n[OK] ALL CRITICAL VALIDATIONS PASSED - Agent is properly configured!")
print("="*60)


╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ You are a Production-Grade Itinerary Revision Agent (v2.0) with advanced error recovery and optimization             ║
║ capabilities.                                                                                                        ║
║ ## Mission Critical Objective                                                                                        ║
║ Revise the provided travel itinerary to achieve `success: true` on ALL evaluation criteria in MINIMUM iterations.    ║
║ ## Operational Phases (Follow in STRICT ORDER)                                                                       ║
║ ### Phase 1: DIAGNOSIS (Iterations 1-2)                                                                              ║
║ **Goal**: Understand ALL problems before attempting fixes                                                            ║
║ **Required Actions**:        

In [43]:
# Now let's run the ReAct cycle multiple times to get the revised itinerary.
# Note: If this takes more than a few minutes, then the agent may be stuck in a loop.
# Examine the traces to understand where it is failing and see if adjusting the system prompt helps.
# Since LLMs are stochastic, you will get different results each time you run this cell.
# No changes needed here.

itinerary_revision_agent = ItineraryRevisionAgent()
travel_plan_2 = itinerary_revision_agent.run_react_cycle(
    original_travel_plan=travel_plan_1, max_steps=15,
    model=MODEL,
    client=client,
)

print("[OK] Revised itinerary generated successfully. Congratulations!")



╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ You are a Production-Grade Itinerary Revision Agent (v2.0) with advanced error recovery and optimization             ║
║ capabilities.                                                                                                        ║
║ ## Mission Critical Objective                                                                                        ║
║ Revise the provided travel itinerary to achieve `success: true` on ALL evaluation criteria in MINIMUM iterations.    ║
║ ## Operational Phases (Follow in STRICT ORDER)                                                                       ║
║ ### Phase 1: DIAGNOSIS (Iterations 1-2)                                                                              ║
║ **Goal**: Understand ALL problems before attempting fixes                                                            ║
║ **Required Actions**:        

In [44]:
# Last let's double check that the revised travel plan passes all evaluation functions.
# No changes needed here.

eval_results_2 = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_2,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

assert eval_results_2.success, f"[ERROR] Read the traces above and modify the system prompt.\n\nFailures: {eval_results_2.failures}"

print("[OK] All evaluation functions passed successfully for the revised travel plan.")

eval_results_2

[OK] Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
[OK] Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
[OK] Traveler Hiro has a match with interest {reading, art} at AgentsVille Twilight Writing Escape
[OK] Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
[OK] Traveler Hiro has a match with interest {music} at Morning Groove Dance Party
[OK] Traveler Hiro has a match with interest {music} at Soundtrack Picnic: Lunchtime Movies & Melodies
[OK] Activity Serve & Savor: Tennis and Taste Luncheon compatible with clear
[OK] Activity AgentsVille Twilight Writing Escape compatible with clear
[OK] Activity Palette & Palate: Art Meets Cooking Experience compatible with partly cloudy
[OK] Activity Morning Groove Dance Party compatible with partly cloudy
[OK] 

EvaluationResults(success=True, failures=[], eval_functions=['eval_start_end_dates_match', 'eval_total_cost_is_accurate', 'eval_itinerary_events_match_actual_events', 'eval_itinerary_satisfies_interests', 'eval_total_cost_is_within_budget', 'eval_activities_and_weather_are_compatible', 'eval_traveler_feedback_is_incorporated'])

In [45]:
# Show the final travel plan in a readable format.
# No changes needed here.

from IPython.display import display

for itinerary_day in travel_plan_2.itinerary_days:
    print(f"Date: {itinerary_day.date}")
    print(
        f"Weather: {itinerary_day.weather.condition} ({itinerary_day.weather.temperature}°{itinerary_day.weather.temperature_unit})"
    )

    activities_df = pd.DataFrame(
        [
            activity_recommendation.activity.model_dump()
            for activity_recommendation in itinerary_day.activity_recommendations
        ]
    )
    display(activities_df)

Date: 2025-06-10
Weather: clear (31.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00:00,2025-06-10 13:30:00,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[cooking, tennis]"
1,event-2025-06-10-3,AgentsVille Twilight Writing Escape,2025-06-10 19:00:00,2025-06-10 21:00:00,"The Ink Loft, 12 Quill Lane, AgentsVille","Join fellow writers for an inspiring evening at The Ink Loft, where words flow as freely as the coffee! This writing-themed event welcomes all—novelists, poets, bloggers, or anyone with a passion for storytelling. Set indoors in AgentsVille's coziest lounge, enjoy writing games, group prompts, and opportunities to read your work aloud. Connect, create, and celebrate the art of writing in this creative indoor haven.",15,"[writing, reading, art]"


Date: 2025-06-11
Weather: partly cloudy (34.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-11-3,Palette & Palate: Art Meets Cooking Experience,2025-06-11 18:30:00,2025-06-11 20:30:00,"The Creative Canvas Studio, Artisanal Lane, AgentsVille","Immerse yourself in a colorful evening where art and cooking blend together! At 'Palette & Palate,' participants will begin indoors at The Creative Canvas Studio with a guided session to paint their own culinary-inspired masterpiece. Afterwards, a local chef will lead an interactive cooking class, teaching you how to craft vibrant, edible works of art. Whether you're an art enthusiast, a food lover, or both, this creative night is perfect for socializing and expressing yourself through color and flavor! All materials and ingredients are provided. This event is held indoors and welcomes all experience levels in art and cooking.",25,"[art, cooking]"
1,event-2025-06-11-0,Morning Groove Dance Party,2025-06-11 09:00:00,2025-06-11 10:30:00,"Rhythm Hall, Center Plaza, AgentsVille","Start your day with energy and joy at the Morning Groove Dance Party! This lively event welcomes dancers of all levels to join a vibrant indoor session filled with upbeat music and fun routines. Whether you love modern pop, Latin beats, or classic disco, our dance instructors will guide you to move and groove. Connect with fellow dance lovers in the colorful atmosphere of Rhythm Hall. Perfect for fans of dancing, music, and fitness. Let the rhythm move you! (Indoor event.)",15,"[dancing, music, fitness]"


Date: 2025-06-12
Weather: thunderstorm (28.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-12-1,Soundtrack Picnic: Lunchtime Movies & Melodies,2025-06-12 12:00:00,2025-06-12 13:30:00,"Starlight Amphitheater, AgentsVille","Experience the magic of classic movie scenes paired with live music at the outdoor Starlight Amphitheater! Bring your lunch and relax on the lawn as musicians perform iconic film soundtracks while selected clips light up our open-air screen. Perfect for movie buffs and music lovers alike, this engaging event celebrates both arts in a sunny lunchtime setting. In case of rain, the event will move indoors to the adjacent Harmony Hall. Come for the tunes, stay for the cinematic wonder!",15,"[movies, music]"
1,event-2025-06-12-3,Tech & Film Fusion Night,2025-06-12 19:00:00,2025-06-12 21:30:00,"Virtual Reality Theater, Silicon Plaza, AgentsVille","Dive into an immersive evening where the magic of movies meets the latest in technology! Join fellow movie buffs and tech enthusiasts for a special screening of cutting-edge sci-fi short films, followed by an interactive panel with local filmmakers and VR technologists. Experience the future of entertainment and discuss how technology is transforming the world of cinema. This exciting, indoor event at the Virtual Reality Theater is perfect for anyone interested in technology and movies.",15,"[technology, movies]"


## And, just for fun!

In [ ]:
# And finally, just for fun, let's narrate the trip.
# No changes needed here.

from project_lib import narrate_my_trip

narrate_my_trip(
    vacation_info=vacation_info,
    itinerary=travel_plan_2,
    client=client,
    model=MODEL,  # Optionally, you can change the model here
)


Your upcoming trip to AgentsVille features two travelers, Yuri and Hiro. Yuri is 30 years old and has a passion for tennis, cooking, comedy, and technology. Hiro is 25 and enjoys reading, music, theatre, and art. The trip spans three days, from June 10 to June 12, 2025, with a total planned budget of 130 units and a total itinerary cost of 105. The activities selected reflect the travelers' diverse interests and take into account the weather conditions expected during your stay.

On June 10, the weather is clear and warm, perfect for outdoor and indoor events. The day offers a delightful blend of tennis and culinary experiences with "Serve & Savor: Tennis and Taste Luncheon," an event that combines a friendly outdoor tennis game followed by a hands-on cooking workshop. This event will especially appeal to Yuri’s interests in tennis and cooking while enjoying the beautiful weather. Later in the evening, there is an indoor literary and artistic gathering called the "AgentsVille Twilight Writing Escape" at The Ink Loft, which is ideal for Hiro. This cozy event invites participants to engage in writing games, group prompts, and sharing their stories in a comfortable setting, aligning well with his love for reading and art.

On June 11, the weather is partly cloudy with a warm temperature, making it suitable for mostly indoor activities. The day begins with the energetic "Morning Groove Dance Party," perfect for Hiro’s interest in music and dancing. Later in the evening, both travelers can enjoy the "Palette & Palate: Art Meets Cooking Experience," a creative indoor event that fuses art and cooking. This unique workshop includes painting a culinary-inspired masterpiece followed by an interactive cooking class—perfect for combining Hiro’s art enthusiasm with Yuri’s love of cooking, offering an engaging and social atmosphere.

June 12 brings thunderstorms, so the itinerary wisely includes weather-resistant activities. Around midday, the "Soundtrack Picnic: Lunchtime Movies & Melodies" invites you to enjoy classic movie scenes with live music. While it is an outdoor event, it has a rain-safe plan to move indoors if needed, ensuring comfort despite the weather. In the evening, the "Tech & Film Fusion Night" showcases a special screening of sci-fi short films and an interactive panel on technology's impact on cinema. This indoor event is tailored to Yuri’s fascination with technology and Hiro’s interest in movies, providing an immersive and thought-provoking end to the trip.

Overall, this itinerary thoughtfully balances your interests and expected weather, offering a stimulating mix of outdoor fun, creative workshops, cultural experiences, and tech-savvy entertainment!

## CONGRATULATIONS! 

You have successfully planned a stellar vacation to AgentsVille! Your AI travel agent has demonstrated advanced reasoning techniques, including role-based prompting, chain-of-thought reasoning, ReAct prompting, and feedback loops

Give yourself a pat on the back for completing this project and completing this course!